In [ ]:
!pip install torch

In [ ]:
!pip install torchvision

In [ ]:
import torchvision

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import os
from torchvision import transforms
from  torch.utils.data import DataLoader
from torchvision import models
from sklearn.model_selection import train_test_split
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from torch.utils.data import Dataset

In [ ]:
import torch.nn.functional as F

In [ ]:
import numpy as np

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split


data_folder = "/content/drive/MyDrive/Nill_Data"

file_paths = []
labels = []
for root, dirs, files in os.walk(data_folder):
    print(root)
    for file in files:
        file_path = os.path.join(root, file)
        label = os.path.basename(root)
        print(label)
        file_paths.append(file_path)
        labels.append(label)

/content/drive/MyDrive/Nill_Data
/content/drive/MyDrive/Nill_Data/Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Eye
Open Ey

In [ ]:
print(len(labels))

3158


In [ ]:
print(len(file_paths))

3158


In [ ]:
train_data,valid_data,train_labels,valid_labels=train_test_split(file_paths,labels,test_size=0.05,random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels_encoded =  label_encoder.fit_transform(train_labels)
valid_labels_encoded =  label_encoder.fit_transform(valid_labels)

In [ ]:
num_classes = len(label_encoder.classes_)

In [ ]:
print(num_classes)

2


In [ ]:
labels = F.one_hot(labels, num_classes=num_classes)

In [ ]:
labels[0]

tensor([0, 1], device='cuda:0')

In [ ]:
class CustomData(Dataset):
    def __init__(self,data,labels,transform=None):
        self.data=data
        self.labels=labels
        self.transform=transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        img_path=self.data[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
transform = transforms.Compose([
  transforms.Resize((224, 224)),  # expected 224,224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # expected mean and std
])

train_dataset = CustomData(train_data, train_labels_encoded, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
valid_dataset=CustomData(valid_data,valid_labels_encoded,transform=transform)
valid_loader=DataLoader(valid_dataset,batch_size=8,shuffle=False,num_workers=2)

In [260]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=3, kernel_size=5, stride=1, padding=0),
              nn.ReLU(),
            # nn.AvgPool2d(kernel_size=2, stride=2),
            # nn.Conv2d(in_channels=16, out_channels=3, kernel_size=5, stride=1, padding=0),
            #   nn.ReLU(),
            # nn.AvgPool2d(kernel_size=2, stride=2),


        )
    def forward(self,x):
        a1=self.feature_extractor(x)
        return a1

In [261]:
class STN(nn.Module):
    def __init__(self):
        super(STN, self).__init__()
        self.localization = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(128, 512, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(512, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)

        )
        self.fc_loc = nn.Sequential(
            nn.Linear(90, 512),  # Ensure this matches the flattened size of localization output
            nn.ReLU(True),
            nn.Linear(512, 6)  # 6 parameters for the affine transformation matrix
        )
        # Initialize with identity transformation
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def forward(self, x_encoder, x):
        # Apply STN
        xs = self.localization(x_encoder)
        xs = xs.view(xs.size(0), -1)  # Flatten
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)  # Affine transformation matrix
        grid = F.affine_grid(theta, x_encoder.size(), align_corners=True)
        x_transformed = F.grid_sample(x_encoder, grid, align_corners=True)
        return x_transformed



In [279]:
class Drowness(nn.Module):
        def __init__(self, num_classes):
                super(Drowness, self).__init__()
                self.model = LeNet()
                self.stn = STN()
                self.encoder = nn.Sequential(*list(models.resnet18(pretrained=True).children())[:-5])
                self.channel_reshape = nn.Conv2d(64, 3, kernel_size=1)
                self.fc = nn.Sequential(

                    nn.Linear(147,32) ,
                    nn.Relu(),
                    nn.Linear(32,16),
                    nn.Relu(),
                    nn.Linear(16,num_classes)
                )

        def forward(self, x):
                x_encoded = self.encoder(x)
                x_encoded=self.channel_reshape(x_encoded)
                x1 = self.stn(x_encoded, x)
                features = self.model(x1)
                features = torch.flatten(features, 1)
                pred = self.fc(features)
                return pred, x1

In [280]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=Drowness(num_classes) # num of classes is the num of labels
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AttributeError: module 'torch.nn' has no attribute 'Relu'

In [281]:
criterion = nn.CrossEntropyLoss() # cross entropy for multi class classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # the optim on the gd ADAM
epochs=5

In [282]:

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        pred_state, stn = model(images)

        # CrossEntropyLoss combines softmax with the loss calculation
        loss = criterion(pred_state, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {(running_loss/len(train_loader)):.4f}')

    model.eval()
    correct_prediction = 0
    total_prediction = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            pred_state, stn = model(images)
            _, predicted = torch.max(pred_state, dim=1)
            total_prediction += labels.size(0)
            correct_prediction += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_prediction / total_prediction
    print(f'Val Acc: {val_accuracy:.2f}%')


Epoch [1/5], Loss: 0.0923
Val Acc: 93.04%
Epoch [2/5], Loss: 0.0892
Val Acc: 96.20%
Epoch [3/5], Loss: 0.0793
Val Acc: 96.84%
Epoch [4/5], Loss: 0.0619
Val Acc: 96.84%
Epoch [5/5], Loss: 0.0573
Val Acc: 96.84%


In [283]:






torch.save(model.state_dict(),"Model_stn.pth")

In [284]:
test_data_paths=valid_data

In [285]:
def load_image(image_path, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    return image.unsqueeze(0) # this for the batch dimension model expected the batch dimension

In [286]:
import cv2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

model.eval()
model.to(device)



for image_path in test_data_paths:

    image = load_image(image_path, transform).to(device)
    pred_state, stn = model(image)
    pred_probs = torch.nn.functional.softmax(pred_state, dim=1).cpu().detach().numpy()
    predicted_state = np.argmax(pred_probs, axis=1)
    predicted_state_label = label_encoder.inverse_transform(predicted_state)

    image = image.squeeze(0).permute(1, 2, 0).cpu().numpy()
    stn = stn[0].permute(1,2,0).cpu().detach().numpy()  # Move to CPU before converting to NumPy array

    stn = np.clip(stn, 0, 1)

    plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
    plt.imshow(image)
    plt.title('original')

    plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
    plt.imshow(stn)
    plt.title('transformed')

    plt.show()

    print(f'state: {predicted_state_label}')
